# Laboratorio 4

In [1]:
import pandas as pd
from pathlib import Path

Utilizarás el conjunto de datos de pokemon.

In [2]:
pkm = (
    pd.read_csv(Path().resolve().parent / "data" / "pokemon.csv", index_col="#")
    .rename(columns=lambda x: x.replace(" ", "").replace(".", "_").lower())
)
pkm.head()

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


## Ejercicio 1

(1 pto)

Agrupar por `generation` y `legendary` y obtener por grupo:

* Promedio de `hp`
* Mínimo y máximo de `sp_atk` y `sp_def`

In [3]:
(
    pkm.groupby(["legendary","generation"])
    .agg(
    hp_max=("hp", "mean"),
    min_atk=("sp_atk","min"),
    max_atk=("sp_atk","max"),
    min_def=("sp_atk","min"),
    max_def=("sp_atk","max") 
    )
)    

hp_max  min_atk  max_atk  min_def  max_def
legendary generation                                                
False     1            64.612500       15      175       15      175
          2            69.613861       10      165       10      165
          3            64.626761       10      165       10      165
          4            69.740741       10      140       10      140
          5            69.353333       15      145       15      145
          6            66.189189       27      150       27      150
True      1            98.000000       95      194       95      194
          2           103.400000       90      115       90      115
          3            81.666667       50      180       50      180
          4           100.846154       75      150       75      150
          5            96.133333       72      170       72      170
          6            87.500000       81      170       81      170

## Ejercicio 2

(1 pto)

El profesor Oakgueda determinó que una buen indicador de pokemones es: 

$$ 0.2 \, \textrm{hp} + 0.4 \,(\textrm{attack} + \textrm{sp_atk})^2 + 0.3 \,( \textrm{defense} + \textrm{sp_deff})^{1.5} + 0.1 \, \textrm{speed}$$

Según este indicador, ¿Qué grupo de pokemones (`type1`, `type2`) es en promedio mejor que el resto? 

In [4]:
pkm=pkm.assign(
    indic=0.2*pkm.loc(1)["hp"]
    +0.4*(pkm.loc(1)["attack"]+pkm.loc(1)["sp_atk"])**2
    +0.3*(pkm.loc(1)["defense"]+pkm.loc(1)["sp_def"])**1.5
    +0.1*pkm.loc(1)["speed"]
)
pkm.head(10)

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary,indic
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,5577.056076
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,8596.609386
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,14016.273434
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,20873.998535
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,5300.957856
6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False,8723.240797
7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False,15549.513407
8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False,27888.800000
9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False,28497.572507


In [5]:
pkm_filtered =(
pkm.groupby(["type1","type2"])
    .agg(
    indic_mean=("indic", "mean")
    )
)
pkm_filtered.head()

indic_mean
type1 type2                 
Bug   Electric   8586.230121
      Fighting  16415.948423
      Fire      11777.532783
      Flying     9478.877505
      Ghost      5959.055716

In [6]:
pkm_filtered2=pkm_filtered[pkm_filtered == pkm_filtered.max()]
pkm_filtered2.loc[lambda x: x.notnull().all(axis=1)].head()

,,indic_mean
type1,type2,
Ground,Fire,44774.854123


__Respuesta__: En promedio, los pokemones Ground-Fire son mejores según el indicador del profesor Oakgueda.

## Ejercicio 3

(1 pto)

Define una función que escale los datos tal que, si $s$ es una columna: 

$$s\_scaled = \frac{s - \min(s)}{\max(s) - \min(s)}$$

Y luego transforma cada columna numérica agrupando por si el pokemon es legendario o no.

In [7]:
def minmax_scale(s):
    return (s - s.min()) / (s.max()-s.min())


In [8]:
pkm.assign(
    attack=lambda df: df.groupby("legendary")["attack"].apply(minmax_scale),
    defense=lambda df: df.groupby("legendary")["defense"].apply(minmax_scale),
    hp=lambda df: df.groupby("legendary")["hp"].apply(minmax_scale),
    sp_atk=lambda df: df.groupby("legendary")["sp_atk"].apply(minmax_scale),
    sp_def=lambda df: df.groupby("legendary")["sp_def"].apply(minmax_scale),
    speed=lambda df: df.groupby("legendary")["speed"].apply(minmax_scale),
    indic=lambda df: df.groupby("legendary")["speed"].apply(minmax_scale)
)


,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary,indic
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,0.173228,0.244444,0.195556,0.333333,0.214286,0.258065,1,False,0.258065
2,Ivysaur,Grass,Poison,0.232283,0.316667,0.257778,0.424242,0.285714,0.354839,1,False,0.354839
3,Venusaur,Grass,Poison,0.311024,0.427778,0.346667,0.545455,0.380952,0.483871,1,False,0.483871
4,Mega Venusaur,Grass,Poison,0.311024,0.527778,0.524444,0.678788,0.476190,0.483871,1,False,0.483871
5,Charmander,Fire,NaN,0.149606,0.261111,0.168889,0.303030,0.142857,0.387097,1,False,0.387097
...,...,...,...,...,...,...,...,...,...,...,...,...
796,Diancie,Rock,Fairy,0.000000,0.357143,0.722222,0.347222,0.722222,0.000000,6,True,0.000000
797,Mega Diancie,Rock,Fairy,0.000000,0.785714,0.500000,0.763889,0.500000,0.461538,6,True,0.461538
798,Hoopa Confined,Psychic,Ghost,0.300000,0.428571,0.222222,0.694444,0.611111,0.153846,6,True,0.153846


## Ejercicio 4

(1 pto)

El profesor Oakgueda necesita saber cuántos pokemones hay luego de filtrar el dataset tal que el grupo de (`type1`, `type2`) tenga en promedio un indicador (el del ejercicio #2) mayor a 40000.

In [9]:
pkm.groupby(["type1","type2"]).filter(lambda df: df["indic"].mean() > 40000)

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary,indic
#,,,,,,,,,,,,
425,Primal Groudon,Ground,Fire,100,180,160,150,90,90,3,True,44774.854123
799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,44369.690779


__Respuesta:__ Hay solo dos pokemones que cumplen esta condición.

## Bonus Track

Honestamente, ¿te causó risa el chiste del profesor Oakgueda? Responde del 1 al 5, donde 1 es equivalente a _"Me dio vergüenza ajena"_ y 5 a _"Me alegró el día y mi existencia en la UTFSM"_.

__Respuesta__: 4, olvidé que 'oak' se leia como 'owk' y cuando lo leí bien me dio risa